# 91_ML 실습01 - 영화추천2

In [1]:
import pandas as pd
import numpy as np
movies = pd.read_csv('/Users/newrandom/Documents/GitHub/ML_study/Zerobase_part09_머신러닝/실습/Datasets/tmdb_5000_movies.csv')
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
                    # id : id
                        # title : 영화제목
                                # genres : 장르
                                            # vote_average : 평균 평점
                                                        # vote_count : 투표 수
                                                                        # popularity : 인기
                                                                                    # keywords : 키워드
                                                                                                # overview : 영화개요
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
                                                                            # x : movies_df['genres']
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords']][:2]

<ipython-input-1-c33f2a706947>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres'] = movies_df['genres'].apply(literal_eval)
<ipython-input-1-c33f2a706947>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)
<ipython-input-1-c33f2a706947>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ..."


---------------------------

In [ ]:
movies_df['genres'][0], (' ').join(movies_df['genres'][0])

(['Action', 'Adventure', 'Fantasy', 'Science Fiction'],
 'Action Adventure Fantasy Science Fiction')

In [2]:
# genres의 각 단어들을 하나의 문장(띄어쓰기로 구분) 으로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
movies_df.head()

<ipython-input-2-445f329d7444>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))


,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,Action Adventure Crime
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,Action Crime Drama Thriller
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...",Action Adventure Science Fiction


In [6]:
## 문자열로 변환된 genres를 CountVectorize 수행
from sklearn.feature_extraction.text import CountVectorizer
                                            # 유사도를 만들어 보고자 함.

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
                                        # ngram_range : 보다 더 풍부하게 데이터를 보게하는 방법 중 하나
                                        # The lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted. 
                                        # All values of n such such that min_n <= n <= max_n will be used. 
                                        # For example an ngram_range of (1, 1) means only unigrams, (1, 2) means unigrams and bigrams, and (2, 2) means only bigrams. 
                                        # Only applies if analyzer is not callable.

genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


## `코사인 유사도`

![코사인유사도](/Users/newrandom/Documents/GitHub/ML_study/Zerobase_part09_머신러닝/실습/lecture_image/91-1.png)
> - `아무런 관계가 없으면` 0 
> - `반대 방향을 보고 있으면` -1
> - `같은 방향을 보고 있으면` 1

![](/Users/newrandom/Documents/GitHub/ML_study/Zerobase_part09_머신러닝/실습/lecture_image/91-2.png)

### 문장의 유사도 측정을 하는 방법 중 하나인 `코사인 유사도 측정`을 수행

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [12]:
# genre_sim 객체에서 높은 값 순으로 정렬할 수 있다.
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
                                # numpy의 argsort()는 오름차순, 내림차순을 할 수 없다.
                                # 대신 [:, ::-1]을 하면 된다.
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


In [13]:
# 추천 영화를 DataFrame으로 반환하는 함수
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]

    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]

    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
                                        # vector로 바꾸는 라인

    return df.iloc[similar_indexes]

In [14]:
# 실행
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1
